In [4]:
import requests
import os
from datasets import load_dataset
import pandas as pd
import json 

dataset = load_dataset("PatronusAI/financebench")
df = dataset['train'].to_pandas()

Lamini supports finetuning for LLAMA-2, so I'm trying to use their library and docs to finetune in this notebook. Here is the documentation I am refering to: https://lamini-ai.github.io/training/finetuning/#__tabbed_1_1

In [7]:


# Function to convert the dataframe into the specified format and save as a jsonl file
def convert_to_jsonl_and_save(df, file_path):
    with open(file_path, 'w') as file:
        for _, row in df.iterrows():
            entry = {
                "user": row["question"],
                "system": "You are a factual chatbot that answers questions about 10-K documents. You only answer with answers you find in following text, no outside information:" + row["evidence_text"], 
                "output": row["answer"]
            }
            file.write(json.dumps(entry) + '\n')

# Assuming the data is read into a pandas dataframe
# data = pd.read_csv('your_file.csv')

# Define the output file path
output_file_path = 'finetuning_data.jsonl'

# Convert the dataframe to jsonl and save
convert_to_jsonl_and_save(df, output_file_path)


In [9]:
def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

# Read the data from the file
finetuning_data = read_jsonl_file(output_file_path)

In [11]:
from lamini import LlamaV2Runner

llm = LlamaV2Runner()

In [12]:
llm.load_data(finetuning_data)

In [13]:
llm.train()

TypeError: can only concatenate str (not "NoneType") to str

In [15]:
from llama import BasicModelRunner

model = BasicModelRunner("meta-llama/Llama-2-7b-chat-hf") 
model.load_data_from_jsonlines(output_file_path, input_key="user", output_key="output")
model.train(is_public=True)

TypeError: can only concatenate str (not "NoneType") to str

In [16]:
#This is an example, taken straight from the docs but it shows the same error, so I don't think it is an issue with the data


from lamini import Lamini

data = [
    {"input": "What's your favorite animal?", "output": "dog"},
    {"input": "What's your favorite color?", "output": "blue"},
]
llm = Lamini(model_name="meta-llama/Llama-2-7b-chat-hf")
llm.train(data=data)

TypeError: can only concatenate str (not "NoneType") to str